In [1]:
# Instalação das libs
# !pip install yfinance==0.2.41
# !pip install crewai==0.28.8
# !pip install 'crewai[tools]'
# !pip install langchain==0.1.20
# !pip install langchain-openai==0.1.7
# !pip install langchain-community==0.0.38
# !pip install duckduckgo-search==5.3.0

In [2]:
from dotenv import load_dotenv
import json
import os
from datetime import datetime


import yfinance as yf


from crewai import Agent, Task, Crew, Process


from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults

load_dotenv()

c:\Users\karyl\.virtualenvs\ia-na-pratica-rocketseat-qYSlvz-q\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


True

In [3]:
# Criando Tool do Yahoo Finance
def fetch_stock_price(ticket: str):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock


yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
    func=lambda ticket: fetch_stock_price(ticket),
)

In [4]:
# Importando OPENAI LLM - GPT
# A Chave de API da OpenAI foi configurada no ambiente com o nome OPENAI_API_KEY durante desenvolvimento.
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
# Criando Agente
stock_price_analyst = Agent(
    role="Senior stock price Analyst",
    goal="find the {ticket} stock price and analyses trends",
    backstory="""
	You're a highly experienced in analyzing the price of an specific stock
    and make predictions about its future price.
	""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[yahoo_finance_tool],
    allow_delegation=True,
)

In [6]:
# Criando Tarefa do Agente
get_stock_price = Task(
    description="Analyze the stock {ticket} price history and create a trend analyses  of up, down or sideways",
    expected_output="""
	Specify the current trend stock price - up, down or sideways.
    eg. stock= 'APPL, price UP'
	""",
    agent=stock_price_analyst,
)

In [7]:
# Importar a Tool de Search
search_tool = DuckDuckGoSearchResults(backend='news',num_results=10)

In [8]:
news_analyst = Agent(
    role="Stock News Analyst",
    goal="""
	Create a short summary of the market news related to the stock {ticket} company. Specify the current 
    trend - up, down or sideways with the news context. For each request stock specify a number between 0 and 100,
    where 0 is extreme fear and 100 is extreme greed.
	""",
    backstory="""
	You're a highly experienced in analyzing the market trends and news and have tracked assest for more then 10 years.
    You're also master level analyst in the tradicional markets and have deep understanding of human psychology.
    You understand news, tittles and information, but you look at those with a health dose of skepticism.
    You consider also the source of the news articles.
	""",
    verbose=True,
    llm=llm,
    max_iter=10,
    memory=True,
    tools=[search_tool],
    allow_delegation=True,
)

In [9]:
get_news = Task(
    description=f"""
    Take the stock and always include BTC to it (if not request)
    Use the search tool to search each one individually.
    The current date is {datetime.now()}
    Compose the results into a helpful report
    """,
    expected_output="""
	A summary of the overall market and one sentence summary for each request asset.
    Include a fear/greed score for each asset based on the news. Use the format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
	""",
    agent=news_analyst,
)

In [10]:
stock_analyst_writer = Agent(
    role="Senior Stock Analyst Writer",
    goal="""
    Analyse the trends prices and news and write an insightful compelling and informative 3 paragraph long 
    newsletter based on the stock report and price trend.
    """,
    backstory="""
	You're widely accepted as the best stock analyst in the market.
    You understand complex concepts and create compelling stories and narratives that resonate with wider audiences.
    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analysis.
    You're able to hold multiple opinions whe analyzing anything.
	""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    allow_delegation=True
)

In [11]:
write_analyses = Task(
    description="""
	Use the stock price trend and the stock news report to create an analyses and write the newsletter 
    about the {ticket} company that is brief and highlights the most important points.
    Focus on the stock price, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary.
    """,
    expected_output="""
	An eloquent 3 paragraphs newsletter formatted as markdown in an easy readable manner.
    It should contain:
    - 3 bullets executive summary
    - Introduction - set the overall picture and spike up the interest
	- main part provides the meat of the analysis including the news summary and fear/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways
	""",
    agent=stock_analyst_writer,
    context=[get_stock_price,get_news]
)

In [12]:
# Criar grupo de Agentes de IA
crew = Crew(
    agents=[stock_price_analyst,news_analyst,stock_analyst_writer],
    tasks=[get_stock_price,get_news,write_analyses],
    verbose= 2,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
	max_iter=15,
)

In [13]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})

ValidationError: 1 validation error for Agent
verbose
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/bool_parsing